In [1]:
import networkx as nx
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

df_path = os.getcwd() + "\\DATA\\parsed_data_public.parquet"

# Cargar el archivo .parquet

#df_raw = pd.read_parquet("C:\\Users\\tatan\\OneDrive\\Desktop\\Mineria_personal\\parsed_data_public.parquet")
df_raw = pd.read_parquet(df_path)


In [2]:
#filtrado inicial de df sin columnas q

# Expresión regular para encontrar columnas que comienzan con 'q' seguida de un número
pattern = r'^q\d+'

# Filtrar las columnas que NO cumplen con el patrón
columnas_filtradas = [col for col in df_raw.columns if not re.match(pattern, col)]

# Mostrar las columnas filtradas
df = df_raw[columnas_filtradas]
df.columns

Index(['Unnamed: 0', 'p_conf', 'd_astrology_seriosity', 'p_laidback',
       'd_education_phase', 'p_drug', 'd_gender', 'p_lit', 'p_progress',
       'd_income', 'p_roman', 'p_dominant', 'd_religion_type', 'd_drugs',
       'p_polit', 'p_pure', 'lf_want', 'd_smokes', 'p_manners', 'p_submissive',
       'p_inde', 'p_kinky', 'p_opti', 'p_sloppy', 'p_indie', 'p_introvert',
       'd_astrology_sign', 'd_country', 'p_arro', 'p_ambi', 'd_drinks',
       'p_cool', 'p_trusting', 'd_age', 'p_thrift', 'd_ethnicity', 'p_organ',
       'p_sexdrive', 'd_offspring_current', 'd_bodytype', 'p_honest',
       'lf_max_age', 'lf_for', 'p_expsex', 'p_artsy', 'p_scien', 'd_job',
       'd_orientation', 'd_languages', 'p_spon', 'p_explife', 'd_relationship',
       'd_offspring_desires', 'p_greed', 'p_capi', 'p_giving', 'p_explove',
       'lf_location', 'p_convenmoral', 'p_aggre', 'p_kind', 'p_extro',
       'p_friendstrangers', 'p_oldfash', 'p_comp', 'p_lovedri', 'p_sprit',
       'lf_min_age', 'p_passion

In [ ]:
print(df.shape)
df["lf_for"].value_counts()

(68371, 85)


lf_for
For  new friends, long-term dating, short-term dating                24542
For  new friends                                                     14803
For  new friends, long-term dating                                    8300
For  new friends, long-term dating, short-term dating, casual sex     5813
For  long-term dating, short-term dating                              4520
For  long-term dating                                                 3233
For  new friends, short-term dating                                   1496
For  new friends, short-term dating, casual sex                       1168
For  long-term dating, short-term dating, casual sex                   787
For  new friends, casual sex                                           470
For  short-term dating                                                 399
For  new friends, long-term dating, casual sex                         316
For  casual sex                                                        255
For  short-term da

: 

In [4]:
#filtramos el df
columnas_filtro = [col for col in df.columns if col[0:2]=="lf"] + ["d_age","d_country","gender2","d_relationship"]

# Col relevantes
df_filtrado = df[columnas_filtro]

# Filas en blancos
df_filtrado = df_filtrado[df_filtrado.isna().sum(axis=1) == 0]

# Generos sencillos
df_filtrado = df_filtrado[df_filtrado["lf_want"].isin(["Women","Men","Everyone"])]
df_filtrado["lf_want"] = df_filtrado["lf_want"].replace("Men","Man")
df_filtrado["lf_want"] = df_filtrado["lf_want"].replace("Women","Woman")

# Paises comunes
top_paises=df["d_country"].value_counts().head(25).index
df_filtrado = df_filtrado[df_filtrado["d_country"].isin(top_paises)]

# Solterx
df_filtrado = df_filtrado[df_filtrado["d_relationship"]=="Single"]


In [5]:
def w(i,j):
    X=df_filtrado.iloc[i]
    Y=df_filtrado.iloc[j]
    
    # AGE
    X_AGE= int((X["lf_min_age"] <= Y["d_age"]) and (X["lf_max_age"] >= Y["d_age"]))
    Y_AGE= int((Y["lf_min_age"] <= X["d_age"]) and (Y["lf_max_age"] >= X["d_age"]))
    AGE = X_AGE*Y_AGE

    # WANT
    if X["lf_want"] == "Everyone":
        X_WANT=1
    else:
        X_WANT = int(X["lf_want"] == Y["gender2"])

    if Y["lf_want"] == "Everyone":
        Y_WANT=1
    else:
        Y_WANT = int(Y["lf_want"] == X["gender2"])

    WANT= X_WANT * Y_WANT
    
    # LOCATION
    if (X["lf_location"] == "Near me") or (Y["lf_location"] == "Near me"):
        LOC=int(X["d_country"]==Y["d_country"])
    else:
        LOC=1    

    # FOR
    x_list = X["lf_for"].split(", ")
    y_list = Y["lf_for"].split(", ")
    e_for = len(set(x_list) & set(y_list))
    if e_for > 0:
        FOR = 1
    else:
        FOR = 0
    
    return bool(AGE*WANT*LOC*FOR)